# Workshop Part 3: Solving the Knapsack Problem with QUBO

Welcome to the final hands-on exercise! We will now tackle another classic optimization problem: the **Knapsack Problem**, framed as a harbor logistics challenge.

### The Problem
Imagine you are a harbor logistics planner. A cargo ship is ready for departure, and to maintain stability and balance, it must be loaded with a specific **exact total weight**. You have a set of available shipping containers, each with its own **weight** (in tons) and a **shipping value** (representing its priority).

Your goal is to select the combination of containers that **maximizes the total shipping value** while ensuring the **total weight of the cargo is exactly equal** to the ship's target tonnage.

This is a simplified version of the Knapsack problem where we must hit the capacity exactly.

## Step 1: Setup and Dependencies

First, we import the necessary libraries. We'll use `matplotlib.patches` to help draw our containers.

In [1]:
# Core scientific computing and plotting libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# pyqubo for building the QUBO model
from pyqubo import Array, Constraint, Placeholder

# neal for the simulated annealing solver
import neal

print("Libraries imported successfully!")

Libraries imported successfully!


## Step 2: Problem Definition

Let's define our available containers and the ship's target tonnage. We've chosen a `target_tonnage` that is achievable by summing up a subset of the container weights.

In [2]:
# Define the containers with their weight (in tons) and shipping value
containers = {
    'Automotive':    {'weight': 8, 'value': 15},
    'Chemicals':     {'weight': 5, 'value': 9},
    'Electronics':   {'weight': 3, 'value': 7},
    'Textiles':      {'weight': 4, 'value': 8},
    'Produce':       {'weight': 2, 'value': 4}
}

# The ship has a fixed weight capacity that we must fill exactly
target_tonnage = 10

container_names = list(containers.keys())
n_containers = len(container_names)

weights = [containers[c]['weight'] for c in container_names]
values = [containers[c]['value'] for c in container_names]

print(f"Problem defined for {n_containers} containers with a target tonnage of {target_tonnage} tons.")

Problem defined for 5 containers with a target tonnage of 10 tons.


## Step 3: QUBO Formulation

Now we translate the problem into a QUBO model. This involves defining our binary variables, the objective function, and the constraint.

### 3.1 Decision Variables

We need a binary variable $x_{i}$ for each container:
- **1** if we load container `i` onto the ship
- **0** if we do not load container `i`

**Your task is to create this array of variables.**

In [ ]:
# --- Your code goes here --- #
# Create a 1D array of binary variables called 'x'.
# It should have a shape equal to the number of containers (n_containers).
x = # ...

### 3.2 The Objective Function

Our goal is to **maximize** the total shipping value. However, QUBO solvers are designed to **minimize** a function. We can easily switch from maximization to minimization by minimizing the *negative* of our value function.

**Your task is to build this objective function.**

In [ ]:
# --- Your code goes here --- #
# Create a variable 'objective_func'.
# Calculate the sum of (value * x) for each container.
# Remember to make the whole expression negative to maximize the value.
objective_func = # ...

### 3.3 The Constraint

We have one strict rule: **The total weight of the loaded containers must be exactly equal to the target tonnage.**

We model this with a penalty term. The sum of the weights of the chosen containers must equal the `target_tonnage`.

**Your task is to build this constraint function.**

In [ ]:
# Use a Placeholder for the penalty weight
P = Placeholder("penalty_weight")

# --- Your code goes here --- #
# Create a variable 'constraint_func'.
# It should be a Constraint that penalizes the difference between the sum of selected weights
# and the target_tonnage.
constraint_func = # ...

### 3.4 The Final Hamiltonian

Now we combine the objective and the constraint into our final model.

**Your task is to combine the objective and constraint.**

In [ ]:
# --- Your code goes here --- #
# Construct the final Hamiltonian 'H' by adding the objective_func
# and the constraint_func (multiplied by the penalty 'P').
H = # ...

## Step 4: Compile and Solve the QUBO

With our Hamiltonian defined, we compile it into a QUBO matrix and send it to the simulated annealer to solve.

**Your task is to complete the solving process.**

In [ ]:
# --- Your code goes here --- #

# 1. Compile the model 'H'
model = # ...

# 2. Define the penalty value in a feed_dict.
# A value of 1.0 is a good starting point.
feed_dict = # ...

# 3. Convert the compiled model to a QUBO dictionary
qubo, offset = # ...

# 4. Create a sampler and solve the QUBO
sampler = # ...
sampleset = # ...

# 5. Decode the best solution found
decoded_sample = # ...

print(f"Solver finished. Lowest energy found: {decoded_sample.energy}")

## Step 5: Interpret and Visualize the Solution

Finally, we parse the solver's output to determine which containers to load onto the ship and visualize the result.

**Your task is to write the code to interpret and visualize the solution.**

In [ ]:
def plot_ship_loading(all_containers, selected_containers):
    """Visualizes the container loading solution."""
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    
    # --- Plot 1: Available Containers ---
    ax[0].set_title('Available Containers at Harbor', fontsize=14)
    ax[0].set_xlim(0, 10)
    ax[0].set_ylim(0, 10)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    y_pos = 8.5
    for name, data in all_containers.items():
        ax[0].add_patch(Rectangle((1, y_pos - 1), 2, 1.5, facecolor='skyblue'))
        text = f"{name}\nValue: {data['value']}\nWeight: {data['weight']}t"
        ax[0].text(4, y_pos, text, va='center', fontsize=10)
        y_pos -= 2
        
    # --- Plot 2: Loaded Ship ---
    ax[1].set_title('Optimal Cargo Loaded on Ship', fontsize=14)
    ax[1].set_xlim(0, 10)
    ax[1].set_ylim(0, 10)
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    
    # Draw ship hull
    ax[1].add_patch(plt.Polygon([[1, 1], [9, 1], [8, 4], [2, 4]], closed=True, color='gray'))
    ax[1].text(5, 2.5, 'CARGO SHIP', ha='center', color='white', weight='bold')

    # Draw loaded containers
    x_pos = 2.2
    for name in selected_containers:
        ax[1].add_patch(Rectangle((x_pos, 4), 1, 1.5, facecolor='gold', edgecolor='black'))
        ax[1].text(x_pos + 0.5, 4.75, name[:4], ha='center', va='center', fontsize=8, weight='bold')
        x_pos += 1.2

    plt.show()

# --- Your code goes here --- #

# Check for broken constraints in the decoded_sample.
# If there are no broken constraints:
#  1. Create an empty list called 'selected_containers'.
#  2. Loop through the containers to find which x[i] is 1.
#  3. Populate the list with the names of the selected containers.
#  4. Print the results (the names, total value, and total weight).
#  5. Call plot_ship_loading(containers, selected_containers) to see your result!
#
# If there are broken constraints, print an error message.